Imports and one article's text

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
from spacy.lang.en import English
from string import punctuation

import spacy
spacy.load('en_core_web_sm')

import scispacy
import en_core_sci_sm

from scispacy.linking import EntityLinker

url = "https://www.bbc.com/news/health-61553822"
response = requests.get(url)
#print(response)

html = response.content
#print(html)
soup = bs(html,"lxml")

article = ",".join([el.text for el in soup.find_all("div", [re.compile('.*RichTextContainer.*')], recursive=True)])
#article = [el.text for el in soup.find_all("div", [re.compile('.*RichTextContainer.*')], recursive= True)]
print(article)



Monkeypox can be contained in countries outside of Africa where the virus is not usually detected, the World Health Organization (WHO) says.,More than 100 cases of the virus - which causes a rash and a fever - have been confirmed in Europe, the Americas and Australia.,That number is expected to rise still, but experts say the overall risk to the broader population is very low.,The virus is most common in remote parts of Central and West Africa.,"This is a containable situation," the WHO's emerging disease lead Maria Van Kerkhove said at a news conference on Monday.,"We want to stop human-to-human transmission. We can do this in the non-endemic countries," she added - referring to recent cases in Europe and North America.,What is monkeypox and how do you catch it? Monkeypox: Time to worry or one to ignore?,The virus has now been detected in 16 countries outside Africa.,Despite being the largest outbreak outside of Africa in 50 years, monkeypox does not spread easily between people and e

In [2]:

url = "https://www.bbc.co.uk/sitemaps/https-sitemap-uk-archive-42.xml"
req = requests.get(url)

#print(req.text)

match = re.findall("<loc>(.*health.*)</loc>", req.text)
print(match[:50])

['https://www.bbc.co.uk/news/health-61321501', 'https://www.bbc.co.uk/news/health-61332060', 'https://www.bbc.co.uk/news/health-61335569', 'https://www.bbc.co.uk/news/health-61334099', 'https://www.bbc.co.uk/news/health-61327778', 'https://www.bbc.co.uk/news/health-61349196', 'https://www.bbc.co.uk/news/health-61349651', 'https://www.bbc.co.uk/news/health-61387871', 'https://www.bbc.co.uk/news/health-61309962', 'https://www.bbc.co.uk/news/health-61398886', 'https://www.bbc.co.uk/news/health-61384269', 'https://www.bbc.co.uk/news/health-61340244', 'https://www.bbc.co.uk/news/health-61335711', 'https://www.bbc.co.uk/news/health-61413576', 'https://www.bbc.co.uk/news/health-61377521', 'https://www.bbc.co.uk/news/health-61423500', 'https://www.bbc.co.uk/news/health-61436454', 'https://www.bbc.co.uk/news/health-61468862', 'https://www.bbc.co.uk/news/health-61441539', 'https://www.bbc.co.uk/news/health-61392698', 'https://www.bbc.co.uk/news/health-61426099', 'https://www.bbc.co.uk/news/healt

In [3]:
list_of_extracted_texts = []
nlp = English()
nlp = spacy.load("en_core_web_sm")
nlp = en_core_sci_sm.load()
nlp = spacy.load("en_core_sci_sm")

nlp.add_pipe("scispacy_linker", config={"linker_name": "umls", "max_entities_per_mention": 5})

for url in match[:50]:
    req = requests.get(url)
    soup = bs(req.content, "lxml")

    list_of_extracted_texts = ",".join([el.text for el in soup.find_all("div", [re.compile('.*RichTextContainer.*')], recursive=True)])
    #puntuation  removal
    clean_list_of_extracted_texts = re.sub(f"[{re.escape(punctuation)}]", "", list_of_extracted_texts)
    #list_of_extracted_texts = [el.text for el in soup.find_all("div", [re.compile('.*RichTextContainer.*')], recursive=True)]
    #print(clean_list_of_extracted_texts)

    # shouldn't this give the first list element i.e. the text of the first article ?? it doesn't. It rather outputs the first letter of each article!
    #print(list_of_extracted_texts[0])


    #both the for loops give an letter-wise break up of the articles, as the list is already broken up into separate articles, we dont even need this maybe
    #for text in list_of_extracted_texts:
     #    print(text)
    #for i in range(len(list_of_extracted_texts)):
     #   print(list_of_extracted_texts[i])


    # Doesn't return anything
    #print(soup.find(property="articleBody"))
    #list_of_extracted_texts.append([el.text for el in soup.find_all("div",[re.compile('.*RichTextConatianer.*')], recursive=True)])   
#print(list_of_extracted_texts)

#TOKENIZATION
    doc = nlp(clean_list_of_extracted_texts)
    #for token in doc:
       # print(token)

#POS tagging, etc.
   # fmt_str = "{:<15}| {:<10}| {:<8}| {:<8}"
    #print(fmt_str.format("token","pos", "label","parent"))
    #for token in doc:
     #   print(fmt_str.format(token.text, token.pos_, token.ent_type_, token.head.text))

#With entity type
    #fmt_str = "{:<15}| {:<6}| {:<7}| {:<8}"
    #print(fmt_str.format("token", "pos", "label", "parent"))
    #for token in doc:
     #   print(fmt_str.format(token.text, token.pos_, token.ent_type_, token.head.text))

#Integrating UMLS
    
    fmt_str = "{:<15}| {:<15}| {:<6}"
    print(fmt_str.format("Entity", "Concept ID", "Score"))

    #alter the ents[value] to see other entitities
    entity = doc.ents[7]
    for kb_entry in entity._.kb_ents:
        cui = kb_entry[0]
        match_score = kb_entry[1]
        print(fmt_str.format(entity.text, cui, match_score))


    #linker = nlp.get_pipe("scispacy_linker")
    #fmt_str = "{:<35}| {:<10}| {:<32}| {:<20}"
    #print(fmt_str.format("Entity", "1st CUI", "Canonical Name", "Definition"))
    #for entity in doc.ents:
     #   first_cuid = entity._.kb_ents[0][0]
      #  kb_entry = linker.kb.cui_to_entity[first_cuid]
       # print(fmt_str.format(entity.text, first_cuid, kb_entry.canonical_name, kb_entry.definition[0:15] + "..."))    
    




c:\Apps\Anaconda\envs\BBC Health data analysis\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
c:\Apps\Anaconda\envs\BBC Health data analysis\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
c:\Apps\Anaconda\envs\BBC Health data analysis\lib\site-packages\scispacy\candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequence

Entity         | Concept ID     | Score 
Entity         | Concept ID     | Score 
Entity         | Concept ID     | Score 
Entity         | Concept ID     | Score 
employees      | C1527116       | 0.9999999403953552
employees      | C0599987       | 0.8699978590011597
employees      | C0031222       | 0.744744062423706
employees      | C3243950       | 0.734083890914917
employees      | C4277725       | 0.7326488494873047
Entity         | Concept ID     | Score 
countries      | C0454664       | 1.0   
countries      | C0004715       | 0.7558684349060059
countries      | C0079170       | 0.7101070284843445
Entity         | Concept ID     | Score 
population     | C0032659       | 1.0   
population     | C1257890       | 1.0   
population     | C0086409       | 0.8412508368492126
population     | C5205739       | 0.8050444722175598
population     | C1518563       | 0.7780501246452332
Entity         | Concept ID     | Score 
Entity         | Concept ID     | Score 
millions       | C188

IndexError: tuple index out of range